In [1]:
import dew_gwdata as gd
import pandas as pd

In [2]:
db = gd.sageodata()

In [49]:
db.query(
    """
(select distinct strat_unit_no, hydro_subunit_code from
dhdb.wa_hydrostrat_subint_vw 
where strat_unit_no = 4062
)
"""
)

,strat_unit_no,hydro_subunit_code
0,4062,Aq
1,4062,CB


In [73]:
sql = """
SELECT dh.drillhole_no                             AS dh_no, 
       dh.unit_no                                  AS unit_long, 
       To_char(dh.map_100000_no) 
       || '-' 
       || To_char(dh.dh_seq_no)                    AS unit_hyphen, 
       Trim(To_char(dh.obs_well_plan_code)) 
       || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no, 
       dh.dh_name, 
       dh.amg_easting                              AS easting, 
       dh.amg_northing                             AS northing, 
       dh.amg_zone                                 AS zone, 
       dh.neg_lat_deg_real                         AS latitude, 
       dh.long_deg_real                            AS longitude, 
       summ.aq_subaq                               AS current_aquifer, 
       aqmon.constrn_date                          AS aquifer_mon_from, 
       su.map_symbol 
       || CASE 
            WHEN hs_subint.hydro_subunit_code IS NOT NULL THEN '(' 
                                                               || 
            hs_subint.hydro_subunit_code 
                                                               || ')' 
            ELSE '' 
          END                                      AS aquifer_mon, 
       su.strat_name                               AS strat_name, 
       hs_subunit.hydro_subunit_desc               AS aquifer_desc, 
       aqmon.comments, 
       aqmon.created_by, 
       aqmon.creation_date, 
       aqmon.modified_by, 
       aqmon.modified_date 
FROM   dhdb.dd_dh_aquifer_mon_vw aqmon 
       inner join dhdb.dd_drillhole_vw dh 
               ON aqmon.drillhole_no = dh.drillhole_no 
       inner join dhdb.dd_drillhole_summary_vw summ 
               ON aqmon.drillhole_no = summ.drillhole_no 
       inner join dhdb.st_strat_unit_vw su 
               ON aqmon.strat_unit_no = su.strat_unit_no 
       left outer join (SELECT DISTINCT strat_unit_no, 
                                        hydro_subunit_code 
                        FROM   dhdb.wa_hydrostrat_subint_vw) hs_subint 
                    ON aqmon.strat_unit_no = hs_subint.strat_unit_no 
                       AND aqmon.hydro_subunit_code = 
                           hs_subint.hydro_subunit_code 
       left outer join dhdb.wa_hydrostrat_subunit_vw hs_subunit 
                    ON hs_subint.strat_unit_no = hs_subunit.strat_unit_no 
                       AND hs_subint.hydro_subunit_code = 
                           hs_subunit.hydro_subunit_code 
WHERE  aqmon.drillhole_no IN ( 8884 ) 
ORDER  BY dh_no, 
          aquifer_mon_from 
"""
df = db.query(sql)
df.pipe(gd.cleanup_columns)

,well_id,current_aquifer,aquifer_mon_from,aquifer_mon,strat_name,aquifer_desc,comments,created_by,creation_date,modified_by,modified_date
0,SQR076,Qpcb,1963-05-14,Tbe(Aq),Poelpena Formation,Tertiary sand aquifer,None,DHDB,1998-02-19 17:36:03,DHDB,2018-09-24 09:09:39
1,SQR076,Qpcb,1992-11-19,Qpcb,Bridgewater Formation,None,Added 08/04/2020,KINVERARITY,2020-04-08 20:20:10,None,NaT


In [96]:
sql = """
select dh_no, unit_long, unit_hyphen,obs_no,dh_name,easting,northing, zone, latitude,longitude,current_aquifer,
aquifer_mon_from, aquifer_mon, strat_name, aquifer_desc, comments, created_by,creation_date,modified_by,modified_date
from (SELECT dh.drillhole_no as dh_no,
       dh.unit_no                                  AS unit_long, 
       To_char(dh.map_100000_no) 
       || '-' 
       || To_char(dh.dh_seq_no)                    AS unit_hyphen, 
       Trim(To_char(dh.obs_well_plan_code)) 
       || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no, 
       dh.dh_name, 
       dh.amg_easting                              AS easting, 
       dh.amg_northing                             AS northing, 
       dh.amg_zone                                 AS zone, 
       dh.neg_lat_deg_real                         AS latitude, 
       dh.long_deg_real                            AS longitude, 
       summ.aq_subaq                               AS current_aquifer, 
       aqmon.constrn_date                          AS aquifer_mon_from, 
       su.map_symbol 
       || CASE 
            WHEN hs_subint.hydro_subunit_code IS NOT NULL THEN '(' 
                                                               || 
            hs_subint.hydro_subunit_code 
                                                               || ')' 
            ELSE '' 
          END                                      AS aquifer_mon, 
       su.strat_name                               AS strat_name, 
       hs_subunit.hydro_subunit_desc               AS aquifer_desc, 
       aqmon.comments, 
       aqmon.created_by, 
       aqmon.creation_date, 
       aqmon.modified_by, 
       aqmon.modified_date 
FROM   dhdb.dd_dh_aquifer_mon_vw aqmon 
       inner join dhdb.dd_drillhole_vw dh 
               ON aqmon.drillhole_no = dh.drillhole_no 
       inner join dhdb.dd_drillhole_summary_vw summ 
               ON aqmon.drillhole_no = summ.drillhole_no 
       inner join dhdb.st_strat_unit_vw su 
               ON aqmon.strat_unit_no = su.strat_unit_no 
       left outer join (SELECT DISTINCT strat_unit_no, 
                                        hydro_subunit_code 
                        FROM   dhdb.wa_hydrostrat_subint_vw) hs_subint 
                    ON aqmon.strat_unit_no = hs_subint.strat_unit_no 
                       AND aqmon.hydro_subunit_code = 
                           hs_subint.hydro_subunit_code 
       left outer join dhdb.wa_hydrostrat_subunit_vw hs_subunit 
                    ON hs_subint.strat_unit_no = hs_subunit.strat_unit_no 
                       AND hs_subint.hydro_subunit_code = 
                           hs_subunit.hydro_subunit_code ) g
WHERE  'Tbe(Aq)' in g.aquifer_mon
"""
df = db.query(sql)
# df
df[df.obs_no == "SQR076"].pipe(gd.cleanup_columns)

c:\devapps\kent\projects\dew_gwdata\dew_gwdata\utils.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["well_id"] = df.apply(apply_well_id, axis="columns")


,well_id,current_aquifer,aquifer_mon_from,aquifer_mon,strat_name,aquifer_desc,comments,created_by,creation_date,modified_by,modified_date
100,SQR076,Qpcb,1963-05-14,Tbe(Aq),Poelpena Formation,Tertiary sand aquifer,None,DHDB,1998-02-19 17:36:03,DHDB,2018-09-24 09:09:39


In [ ]:
join dhdb.wa_hydrostrat_subint_vw hs_subint on aqmon.hydro_subunit_code = hs_subint.hydro_subunit_code